In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('labour.csv')

 - Here I am aiming to save my dataset in mysql database and use it in my project. I will use the sqlalchemy library for this.
 - I will have to define an engine() for this process, but before that, two different situations arise as a connection string. This is because I will first connect to my database and create a database with a name I want. Then, in order to save my dataset to the database, I need to reconnect my engine() connection with the name of the database I created.
 - It looks like a code mess, but whatever method I tried, I couldn't succeed. For the reasons I explained in the previous article, I had to manage engine() with two different connection strings in this way.
 - Instead of using a local mysql solution, I created an azure mysql database, which is one of the free services specially defined for students, using the azure portal of the microsoft company and I will use it in my project.

In [4]:
from sqlalchemy import create_engine, text

user = 'ca2023'
password = 'CCT2023!'
host = 'cctmysql.mysql.database.azure.com'
port = 3306
database = 'cct_db'

engine_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}'
db_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(engine_url, echo=True)

with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database};"))
    conn.close()

    

2023-05-10 23:50:40,338 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-10 23:50:40,339 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:40,828 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-10 23:50:40,829 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:41,078 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-10 23:50:41,079 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:41,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-10 23:50:41,406 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS cct_db;
2023-05-10 23:50:41,406 INFO sqlalchemy.engine.Engine [generated in 0.16801s] {}
2023-05-10 23:50:41,493 INFO sqlalchemy.engine.Engine ROLLBACK


 - Now that I have created my database, I will send the dataframe to my database using my connection string containing the database for engine().
 - I carry out my process by checking for errors due to the possibility of the current table name being in the database.
 - I am performing my operation using the **to_sql()** function of the pandas library.

In [5]:
engine = create_engine(db_url, echo=True)
try:
    df.to_sql('labour', con=engine, if_exists='fail', index=False)
except Exception as ex:
    print(ex)
else:
    print("Table created")

2023-05-10 23:50:44,047 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-10 23:50:44,048 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:44,532 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-10 23:50:44,534 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:44,784 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-10 23:50:44,786 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:45,193 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-10 23:50:45,194 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-10 23:50:45,194 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:45,273 INFO sqlalchemy.engine.Engine ROLLBACK
Table 'labour' already exists.


In [6]:
conn = engine.connect()
sqldf = pd.read_sql_table('labour', conn)

2023-05-10 23:50:46,644 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-10 23:50:46,645 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-10 23:50:46,645 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:46,890 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `labour`
2023-05-10 23:50:46,891 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-10 23:50:47,137 INFO sqlalchemy.engine.Engine SELECT labour.`STRUCTURE`, labour.`STRUCTURE_ID`, labour.freq, labour.indic_bt, labour.nace_r2, labour.s_adj, labour.unit, labour.geo, labour.`TIME_PERIOD`, labour.`OBS_VALUE`, labour.`OBS_FLAG` 
FROM labour
2023-05-10 23:50:47,139 INFO sqlalchemy.engine.Engine [generated in 0.16403s] {}


In [7]:
sqldf.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2005,76.9,None
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2006,76.7,None
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2007,74.1,None
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2008,97.3,None
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2009,103.7,None


In [8]:
sqldf.shape

(9314, 11)

In [9]:
sqldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9314 entries, 0 to 9313
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STRUCTURE     9314 non-null   object 
 1   STRUCTURE_ID  9314 non-null   object 
 2   freq          9314 non-null   object 
 3   indic_bt      9314 non-null   object 
 4   nace_r2       9314 non-null   object 
 5   s_adj         9314 non-null   object 
 6   unit          9314 non-null   object 
 7   geo           9314 non-null   object 
 8   TIME_PERIOD   9314 non-null   int64  
 9   OBS_VALUE     9108 non-null   float64
 10  OBS_FLAG      1717 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 800.6+ KB


<table border="1" cellspacing="0" cellpadding="0">
    <tbody>
        <tr>
            <td width="301" valign="top">
                <p>
                    Column Name
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Description
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    freq
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Frequency of publication of data
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    Indic_bt
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    The type of statistic (employment, production, sales, etc.)
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    nace_r2
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Code showing which industry classification the data relates
                    to
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    s_adj
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Code indicating whether it is seasonally adjusted or
                    unadjusted
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    unit
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    unit of measurement
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    geo
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Country
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    TIME_PERIOD
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Year
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    OBS_VALUE
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Value of data <br/>
                    <br/>
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    OBS_FLAG
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    A code that provides additional information about the
                    accuracy or reliability of the data
                </p>
            </td>
        </tr>
    </tbody>
</table>

In [10]:
sqldf.describe()

,TIME_PERIOD,OBS_VALUE
count,9314.000000,9108.000000
mean,2010.120893,72.166359
std,6.580738,54.231636
min,1990.000000,-67.400000
25%,2005.000000,8.100000
50%,2010.000000,91.200000
75%,2015.000000,106.000000
max,2022.000000,406.500000


In [11]:
sqldf.describe(include="object")

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,OBS_FLAG
count,9314,9314,9314,9314,9314,9314,9314,9314,1717
unique,1,1,1,3,1,2,3,36,5
top,dataflow,ESTAT:STS_COLB_A(1.0),A,WAGE,F,NSA,I15,NL,p
freq,9314,9314,9314,3546,9314,5728,3931,338,866


- I will use the uniqe() function to display what are the unique values ​​in my categorical columns. 
- I will also use tabulate, a simple library of tables, to display values ​​neatly

In [22]:
from tabulate import tabulate

table_data = {
    "indic_bt": sqldf.indic_bt.unique(),
    "s_adj": sqldf.s_adj.unique(),
    "unit": sqldf.unit.unique()
}

table = tabulate(table_data, headers="keys", tablefmt="psql")

print(table)

+------------+---------+--------+
| indic_bt   | s_adj   | unit   |
|------------+---------+--------|
| EMPL       | NSA     | I10    |
| HOWK       | CA      | I15    |
| WAGE       |         | PCH_SM |
+------------+---------+--------+


### I've added a small table explanation to explain what the values ​​for my categorical columns mean, the abbreviation values ​​on the eurostat website
<table border="1" cellspacing="0" cellpadding="0">
    <tbody>
        <tr>
            <td width="204" colspan="2" valign="top">
                <p align="center">
                    Indic_bt
                </p>
            </td>
            <td width="198" colspan="2" valign="top">
                <p align="center">
                    S_adj
                </p>
            </td>
            <td width="199" colspan="2" valign="top">
                <p align="center">
                    unit
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    EMPL
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Employment (number of persons employed)
                </p>
            </td>
            <td width="42" valign="top">
                <p>
                    NSA
                </p>
            </td>
            <td width="156" valign="top">
                <p>
                    Unadjusted data (i.e. neither seasonally adjusted nor
                    calendar adjusted data)
                </p>
            </td>
            <td width="70" valign="top">
                <p>
                    I10
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Index, 2010=100
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    HOWK
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Volume of work done (hours worked)
                </p>
            </td>
            <td width="42" valign="top">
                <p>
                    CA
                </p>
            </td>
            <td width="156" valign="top">
                <p>
                    Calencar adjusted data, not seasonally adjusted data
                </p>
            </td>
            <td width="70" valign="top">
                <p>
                    I15
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Index, 2015=100
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    WAGE
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Gross wages and salaries
                </p>
            </td>
            <td width="42" valign="top">
            </td>
            <td width="156" valign="top">
            </td>
            <td width="70" valign="top">
                <p>
                    PCH_SM
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Percentage change compared to same period in previous year
                </p>
            </td>
        </tr>
    </tbody>
</table>

- Since I will be using the employment data, which is the main subject of my project, I will take the EMPL values in the indic_bt column. 
- Preferably by using the i15 values from the unit column, I will get the records from 2015 of my project where the index value of all countries is 100.

In [32]:
filtered_df = sqldf.loc[(df['indic_bt'] == 'EMPL') & (df['unit'] == 'I15')]

In [33]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
655,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2005,66.4,None
656,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2006,66.2,None
657,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2007,64.0,None
658,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2008,84.1,None
659,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2009,89.6,None


In [38]:
filtered_df.shape

(839, 11)

In [39]:
filtered_df.describe(include="object")

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,OBS_FLAG
count,839,839,839,839,839,839,839,839,70
unique,1,1,1,1,1,1,1,36,4
top,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,FR,e
freq,839,839,839,839,839,839,839,33,35


When I look at the records in the first 5 rows of my filtered dataset, I notice that my index values are not normal. Since I have created a new dataframe by filtering from my existing dataset according to certain criteria, the index values appear as row numbers in my current dataset. I will use the reset_index() function to fix this and apply sequential index values to the dataframe.

In [40]:
filtered_df.reset_index(drop=True, inplace=True)

In [41]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2005,66.4,None
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2006,66.2,None
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2007,64.0,None
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2008,84.1,None
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I15,AL,2009,89.6,None


Now that I have done the necessary filtering according to the categorical columns of my dataset, I can now start my operations with numeric columns. For this, first of all, I want to check if there are any empty or missing values in my rows.